# <a id='toc1_'></a>[Cas pratique sur les jeux vidéos : Diablo](#toc0_)


**Sommaire**<a id='toc0_'></a>    
- [Cas pratique sur les jeux vidéos : Diablo](#toc1_)    
  - [Introdution](#toc1_1_)    
    - [Bases du fonctionnement du jeu](#toc1_1_1_)    
    - [Explication du code source](#toc1_1_2_)    
      - [Initialisation des variables](#toc1_1_2_1_)    
      - [Boucle principale de recherche](#toc1_1_2_2_)    
      - [Vérification d'arrivée](#toc1_1_2_3_)    
      - [Construction du chemin final](#toc1_1_2_4_)    
      - [Copie du chemin inversé](#toc1_1_2_5_)    
      - [Gestion des échecs](#toc1_1_2_6_)    
      - [Retour en cas d'absence de solution](#toc1_1_2_7_)    
      - [Remarque sur les absences de solution](#toc1_1_2_8_)    
    - [Lien vers le code source](#toc1_1_3_)    
    - [Conclusion](#toc1_1_4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Introdution](#toc0_)

Diablo est un jeu vidéo de type hack 'n' slash développé par Blizzard North. La version PC fonctionnant sous Windows a été publiée en janvier 1997 en Amérique du Nord et en Europe par Blizzard Entertainment. Une version pour PlayStation, développée par Electronic Arts, et une version Macintosh sont publiées l'année suivante en mars 1998.

Diablo se déroule dans un univers médiéval-fantastique où le joueur explore des donjons peuplés de créatures qu'il doit combattre pour gagner de l'expérience et des trésors, l'objectif du jeu étant de retrouver Diablo et le tuer. Le jeu est considéré comme ayant réinventé le genre du jeu vidéo de rôle en y introduisant le principe du point and click et en le rendant plus accessible. Diablo a ainsi fortement influencé l'industrie du jeu vidéo, son système de jeu ayant été repris dans de nombreux titres qualifiés de « clones » ou de « Diablo-like ».

Le jeu est extrêmement bien accueilli par les critiques et connaît dès sa sortie un solide succès commercial. En 1997, il décroche ainsi la troisième place des meilleures ventes de jeux vidéo sur PC ; le 29 août 2001, il s'est vendu à plus de 2,5 millions d'exemplaires dans le monde. Par la suite, Blizzard North lance le développement de Diablo II qui sort le 29 juin 2000. Le troisième épisode de la série, Diablo III, annoncé le 28 juin 2008, est sorti le 15 mai 2012. Le quatrième épisode de la série, Diablo IV, est annoncé en novembre 2019 pour une sortie en juin 2023.

### <a id='toc1_1_1_'></a>[Bases du fonctionnement du jeu](#toc0_)

Avant de plonger dans le code du jeu, il est important de connaître un minimum comment 
La carte du jeu est implémentée sous la forme de tuiles, il faut le voir comme un grand graphe avec des sommets, les tuiles qui peuvent être accessibles ou pas, à cause des obstacles, ou alors la présence d'autres entités sur la tuile.
Nous évoquions plus haut la mécanique du point and click, elle consiste à utiliser la souris pour efféctuer des actions simple, comme attaquer par exemple, mais aussi, et surtout, pour déplacer. En effet quand on est sur une tuile, il faut cliquer sur une autre tuile de la carte pour s'y rendre. Il apparaît donc rapidement que la manièer de gérer cette action de déplacement sera primordiale pour les développeur, afin que l'expérience de jeu soit la meilleure.

Le choix des développeurs a donc été de d'implémenter un algorithme mathématique pour rechercher le plus court chemin d'une tuile à une autre. Cet algorithme est donc bien sûr A*, par ailleurs, Diablo I est aujourd'hui considéré comme un des premiers jeux vidéos (avec The Elder Scrolls II) utilisant l'algorithme A* pour la recherche de chemin.

Voici un gif montrant le fonctionnement du déplacement en point and click :

![SegmentLocal](images/diablo.gif "segment")




### <a id='toc1_1_2_'></a>[Explication du code source](#toc0_)

Voici la fonction de recherche de chemin qui est utilisée dans le code pour le calcul du plus court chemin, d'une tuile A vers une tuile B, lorsque l'on clique sur une tuile du jeu, mais aussi lorsque les monstres poursuivent le joueur.

Nous alllons la découper en différentes parties pour faciliter la compréhension

#### <a id='toc1_1_2_1_'></a>[Initialisation des variables](#toc0_)

```c
gdwCurNodes = 0;
path_2_nodes = path_new_step();
pnode_ptr = path_new_step();
gdwCurPathStep = 0;
path_start = path_new_step();
path_start->g = 0;
path_start->h = path_get_h_cost(sx, sy, dx, dy);
path_start->x = sx;
path_start->f = path_start->h + path_start->g;
path_start->y = sy;
path_2_nodes->NextNode = path_start;
```

- Initialise les structures nécessaires au calcul A*
- Crée un nœud de départ avec des coûts (g, h, f) calculés, et le lie comme premier élément de la liste des nœuds à explorer

#### <a id='toc1_1_2_2_'></a>[Boucle principale de recherche](#toc0_)
```c
while ((next_node = GetNextPath())) {
```
- Boucle générale de l'algo
- Parcourt les nœuds en utilisant une méthode pour récupérer le prochain meilleur chemin selon l'algorithme A*


#### <a id='toc1_1_2_3_'></a>[Vérification d'arrivée](#toc0_)
```c
if (next_node->x == dx && next_node->y == dy) {
```
- Si le nœud actuel correspond à la destination, on a trouvé un chemin


#### <a id='toc1_1_2_4_'></a>[Construction du chemin final](#toc0_)
```c
current = next_node;
path_length = 0;
while (current->Parent) {
    if (path_length >= MAX_PATH_LENGTH)
        break;
    pnode_vals[path_length++] = path_directions[3 * (current->y - current->Parent->y) - current->Parent->x + 4 + current->x];
    current = current->Parent;
}
```
- Remonte la chaîne des parents pour reconstruire le chemin, en calculant les directions pour chaque étape
- Si la taille du chemin trouvé est trop longue, on annule et on ne renvoi plus de chemin

#### <a id='toc1_1_2_5_'></a>[Copie du chemin inversé](#toc0_)
```c
if (path_length != MAX_PATH_LENGTH) {
    for (i = 0; i < path_length; i++)
        path[i] = pnode_vals[path_length - i - 1];
    return i;
}
```
- Inverse le chemin pour le mettre dans le bon ordre
- Retourne la longueur du chemin, uniquement si la longueur du chemin n'est pas trop longue

#### <a id='toc1_1_2_6_'></a>[Gestion des échecs](#toc0_)
```c
if (!path_get_path(PosOk, PosOkArg, next_node, dx, dy))
    return 0;
```
- Si on ne peut pas continuer à avancer, on abandonne


#### <a id='toc1_1_2_7_'></a>[Retour en cas d'absence de solution](#toc0_)
```c
return 0;
```
(si la liste des nœuds est vide, il n'y a pas de chemin possible, donc échec)

#### <a id='toc1_1_2_8_'></a>[Remarque sur les absences de solution](#toc0_)
Il est important de noter, que si une solution n'est pas trouvée, cela ne déclenchera pas une erreur critique dans le jeu, en effet, dans ce cas là, le personnage qui effectue le mouvement s'arretera tout simplement.

C'est un choix logique dans le choix d'un jeu vidéo ou le combat est au centre du gameplay, il faut pouvoir permettre au joueur de s'enfuir en cas de difficultés.

### <a id='toc1_1_3_'></a>[Lien vers le code source](#toc0_)

Voici le lien du github comprenant le code étudié : https://github.com/diasurgical/devilution

La fonction ```FindPath``` se trouve dans le fichier ```path.cpp``` du répertoire ```source```

### <a id='toc1_1_4_'></a>[Conclusion](#toc0_)

En conclusion, l'implémentation de l'algorithme A* dans Diablo illustre parfaitement comment une solution algorithmique peut transformer une mécanique de jeu simple en une expérience fluide et engageante. En assurant des déplacements efficaces et réalistes, elle contribue à renforcer l'immersion du joueur dans l'univers du jeu. On peut aussi noter d'autres jeux, comme The leder Scorlls II : Daggerfall, qui ont utilisé l'algorithme A* pour la recherche de chemin, et qui ont été des succès commerciaux et critiques.